<center><h1>SORT and DEEP SORT for Multi-target tracking</h1></center>

Multi-target tracking, or Multiple Object Tracking (MOT), is mainly given an image sequence, finds moving objects in the image sequence, and recognizes moving objects in different frames, that is, given a certain accurate id, Of course, these objects can be arbitrary, such as pedestrians, vehicles, various animals, and so on, and the most research is pedestrian tracking because people are a non-rigid target, and pedestrian detection and tracking has more commercial value in practical applications.

## What is SORT?
The full name of SORT is Simple Online And Real-time Tracking. For the current multi-target tracking, it depends more on its detection performance, and it can be improved by 18.9% by changing the detector. Although this SORT algorithm is just a common algorithm, For example, the combination of Kalman Filter and Hungarian algorithm can match the 2016 SOTA algorithm, and the speed can reach 260Hz, which is 20 times faster than the former.

![Alt text](SORT2.gif)


### Algorithm Overview
Multi-target tracking is regarded as a data association problem. Correlation across detection results is performed in a sequence of video frames. To solve data-related issues, the tracker uses multiple methods to model the motion process and the appearance characteristics of the moving target! But the authors found that these algorithms cannot achieve a trade-off between accuracy and speed, as shown below:

![alt text](Alg.jpg)


Here we did not use any appearance features of the tracked target when tracking the target, but only used the position and size of the detection frame for target motion estimation and data association, and did not perform any re-identification algorithm; Therefore, when the target is lost, it cannot be found. You can only update the ID through detection. This is not in line with the common sense of tracking algorithms and needs to be improved. Of course, this tutorial is mainly about speed! Rather than focusing too much on the robustness of detecting errors!

### Predictive model	
Here we describe the object model, which is the representation and is used to propagate the target's identity to the next frame. Our approximate inter-frame displacements have a linear constant velocity model independent of other objects and camera movement. The state model for each target is as follows:

![alt text](5.jpg)

Where $u$ and $v$ represent the horizontal and vertical coordinates of the center of the target, respectively, and $s$ and $r$ represent the size and proportion of the Box of the target. Note that the aspect ratio should be constant. Therefore, the next three quantities represent the next frame of the prediction. When the detection is associated with the target, the detected bounding box is used to update the target state. The velocity component is optimized by the Kalman method. If no detection is associated with the target, then only a linear velocity model is used.

### The problem of missing targets
If consecutive $T_{lost}$ frames do not achieve an IOU(Intersection over Union) match between the predicted position of the tracked target and the detection frame, the target is considered to have disappeared. It is set in the experiment for  $T_{lost = 1}$  two reasons. One is that the uniform motion assumption is not reasonable. The other is that the author mainly focuses on short-term target tracking. In addition, deleting lost targets early can help improve tracking efficiency. However, the problem arises. In this case, the ID of the target must be frequently switched, which will cause the tracking count to be inaccurate!

### Challenges of object tracking compared to static object detection:

* Re-identification — Connecting an object in one frame to the same object in the subsequent frames
* Appearance and disappearance — Objects can move into or out of the frame unpredictably and we need to connect them to objects previously seen in the video
* Occlusion — Objects are partially or completely occluded in some frames, as other objects appear in front of them and cover them up
* Identity switches — When two objects cross each other, we need to discern which one is which
* Motion blur — Objects may look different due to their own motion or camera motion
* View points — objects may look very different from different viewpoints, and we have to consistently identify the same object from all perspectives
* Scale change — Objects in a video can change scale dramatically, due to camera zoom for example
* Illumination — Lighting changes in a video can have a big effect on how objects look and can make it harder to consistently detect them

## Deep SORT 
With the development of the target detection field in recent years, this tracking-by-detection algorithm has become more and more mainstream in MOT. The previous algorithms, such as the flow network formula and the probabilistic graphical model, deal with the global optimization problem of the entire process. However, it is not suitable for online scenarios, and its target identifier must be available at each time step. More traditional are Hypothetical Tracking (MHT) and Joint Probabilistic Data Correlation Filter (JPDAF). These methods perform frame-by-frame data association. Recently, these methods have been re-understood due to the success of detection problems!

![alt text](6.jpg)

The previous SORT algorithm used a simple Kalman filter to process the correlation of frame-by-frame data and the Hungarian algorithm for association measurement. This simple algorithm achieved good performance at high frame rates. However, because SORT ignores the surface characteristics of the detected object, it is accurate only when the uncertainty of the state estimation of the object is low. In Deep SORT, we use more reliable metrics instead of association metrics, and use CNN networks in large Training and extracting features from large-scale pedestrian datasets has increased the network's robustness to loss and obstacles.

### Track handle and State estimation
* State estimation:  Use an 8-dimensional space to describe the state of the trajectory at a certain moment 
$(u,v,r,h,x^{*}, y^{*}, r^{*},h^{*})$

Represent the position, aspect ratio, height of the center of the bounding box, and the corresponding speed information in image coordinates. A Kalman filter is then used to predict the updated trajectory, which uses a uniform velocity model and a linear observation model. Its observed variables are $(u,v,r,h)$

* <h3>Trajectory processing</h3>: This mainly refers to when the trajectory terminates and when a new trajectory is generated. First, there is a threshold a for each trajectory for recording the time from the last successful matching of the trajectory to the current moment. When the value is greater than the threshold set in advance, the $A_{max}$ trajectory is considered to be terminated. 

* <h3>Problem</h3>
In SORT, we directly use the Hungarian algorithm to solve the correlation between the predicted Kalman state and the new state. Now we need to combine the target motion and surface feature information by fusing these two similar measurement indicators.

* <h3>Motion Metric</h3>
Use the Mahalanobis distance to evaluate the predicted Kalman state and the new state:

![alt text](7.png)

Represents the motion degree of match between the j-th and i-th track detection, which $S_i$ is the covariance matrix of the observation space at the current time obtained from the trajectory kalman filter prediction, $y_i$ a prediction observables trajectory at the current time, $d_j$ the first $j$ Detection status $(u,r,v,h)$


Considering the continuity of motion, detections can be filtered by the Markov distance. Here, the 0.95 quantile of the chi-square distribution is used as the threshold $t^{(1)} =0.4877$ . We can define  a threshold function.
![alt text](8.png)

* <h3>Appearance metric</h3>

When the target motion uncertainty is low, Mahalanobis distance is a good correlation metric, but in practice, such as camera movement, it will cause a large number of Mahalanobis distances that cannot be matched, and this metric will be invalidated. Therefore, we Integrating the second metric, $d_j$ we calculate a surface feature descriptor for each Bounding Box detection box $r_j,|r_j| = 1$ , we will create a gallery to store $L_k = 100$ the descriptors of the latest  trajectory, that is $R_k = \{{r_{k}^{(i)}}\}^{L_k}_{k=1}$ , then we use the i-th trajectory and the j-th trajectory The minimum cosine distance of the trajectory is used as the second measure!

![alt text](9.png)
Of course, we can also use a threshold function to represent
![alt text](10.png)

Then, we merge these two scales into:
![alt text](11.png)

In short, the distance metric is good for short-term prediction and matching, and the apparent information is more effective for long-term lost trajectories. The choice of hyper parameters depends on the specific data set. For example, for data sets with large camera motion, the degree of motion matching is not considered directly.

## Matching cascade

![alt text](12.png)

This tutorial also proposes a cascade matching strategy to improve the matching accuracy, mainly because when an object is occluded for a long time, the uncertainty of Kalman filtering will greatly increase, and the probability of continuous prediction will be dispersed. The variance matrix is a normal distribution. If continuous predictions are not updated, the variance of this normal distribution will become larger and larger. Then, the points that are far away from the mean Euclidean distance may be the same as the points closer to the previous distribution. Mahalanobis distance value.

The above picture describes the cascade matching algorithm!

In the final stage, the author uses the IOU association in the previous SORT algorithm to match the unconfirmed and unmatched trajectories with n = 1. This can alleviate large changes due to apparent mutations or partial occlusion. Of course, there are advantages and disadvantages, and this may cause some newly generated trajectories to be connected to some old trajectories.

## Deep Appearance Descriptor
We builds a deep convolutional neural network to extract the feature information of the target, and uses L2 to normalize the feature projection to a unified hypersphere! The model structure is:
![alt text](13.png)

<h3>Algorithm summary</h3>
Overall, the effect is still obvious. Using CNN-extracted features for matching has greatly reduced the number of ID switches in SORT. The author's experiments have shown that it has reduced about 45%, and has achieved good results in high-rate video streams level!


<center><h1>Kalman Filter</h1></center>

### Introduction

Kalman Filter, when many people first heard the term, they always thought it was a filter. What I want to emphasize here is that Kalman Filter is not filtering, it is a process of information fusion.
So what exactly is Kalman Filter? It has applications in those areas, and what are its basic principles? If you refer to domestic forums or textbooks, you will definitely find that most of them are some complicated formulas and boring abstract descriptions. Here we will explain what Kalman Filter is by explaining it as easily as possible.

### Sneak Preview

As I mentioned earlier, understanding Kalman filtering from definitions and complex formulas is basically impossible for us. Let's simplify the formula:
![alt text](14.png)

The subscript refers to the state. Here we can think of it as the interval, such as 1ms and 2ms. Our aim is to be found in the state when estimates. In the formula is the actual measurement. We must keep in mind that these measurements are not reliable. (If these measurements are reliable, then we don't need to learn kalman filter). Refers to Kalman gain (this is the core point in the entire formula). This is the estimate at the previous state. In the above formula, the only unknown quantity we have is the Kalman gain . We already know the measured values and the last state estimate. Of course, finding the Kalman gain is not easy, but we have ways to solve it. Let's think from another angle. Let us assume that the Kalman gain is 0.5. What do we get? It's a simple average! In other words, we should find a more intelligent Kalman gain coefficient corresponding to each state.
$kk = 1k = 2kXZ_k{K_k}{X_{k-1}}$

The Kalman filter finds the optimal averaging factor for each result state. Somehow, remember something about the past state.

### Step-by-Step guide
Here is a distribution guide to get started with Kalman filtering quickly. 

* Step 1 : Build the model

This is the most important step. First, you must ensure that the conditions of the Kalman filter match your problem.
We remember two typical equations of Kalman filtering:
![alt text](15.png)

The first formula means that each can be represented by a linear random equation. Any one is a linear combination of only the control signal and processing noise of its previous state. In most cases, no control signal is required.
$x_k{x_k}{u_k}{W_{k-1}}{u_k}$

The second formula tells us that any measurement value (here we are not sure whether it is accurate) is a linear combination of the current state signal value and measurement noise, and we default it to follow a Gaussian distribution.
We consider the processing noise and measurement noise in these two formulas to be statistically independent.
The equation coefficients, and is the general form of a matrix. But in most signal processing problems, in the models we use, these coefficients are only numerical values. And when these values change between states, in most cases we can assume that they are constant.
If we determine that our system fits this model (most systems do), then only the estimated noise function and the mean and standard deviation are left. We know that in real life, no signal is purely Gaussian, but we can make assumptions by approximation. 
${W_{k-1}}u_k$

This is not a big problem, as we will see that the Kalman filter algorithm attempts to converge to the correct estimate even if the Gaussian noise parameter estimates are poor.
Here we must always keep in mind: "The more accurate you estimate the noise parameters, the better estimates you will get."

* Step 2: Start Processing

If you successfully fit your model into a Kalman filter, the next step is to determine the necessary parameters and your initial values.
We have two different sets of equations: time update (prediction) and measurement update (calibration). Both sets of equations are applied in the first state $k$ .
![alt text](16.png)

We build the model in step one, so we know the matrices A, B, and H. Most likely, they are all numeric constants. And they most likely all have the value 1.
I suggest you rewrite these formulas and see how to simplify them.
Above formula the most painful thing is to determine the R and Q. R is also relatively easy to find, because, in general, we are relatively certain about the noise in the environment. But finding Q is not obvious. At this stage, we cannot give you a specific method.

* Step3: Iteration

After we collected all the information, we needed, we started the process and now we can iterate the estimates. We have to keep in mind that the estimation of the previous state will be the input to the estimation of the current state
![alt text](17.png)

Here, the prior estimate refers to a rough estimate before the measurement update calibration. Here we use the prior value in the measurement update formula.
In the measurement update formula, we find an estimate of x at state k. And found the value needed for state k + 1 estimation.
The Kalman gain we calculated is not needed in the next iteration step. This is the hidden, mysterious, and most important part of a set of equations.
The value we evaluate during the measurement update phase is also called the posterior value. It also makes sense.


### A simple example for understanding
If you are still confused here, then we will find an example to deepen our understanding.
Now let's do estimate a scalar random constant, such as a "voltage reading" from a signal source. Suppose it has a constant V voltage, but our reading will be inaccurate, it may read high, or it may read low. We assume that the standard deviation of the measurement noise is 0.1 V.
Now let's build our model:

![alt text](18.png)

As I said before, we can simplify the equations into very simple forms.

•First, we have a problem with one-dimensional signals, so each coefficient in our model is a numerical value, not a matrix.

•We have no control signals. $u_k$

•Because the signal is a constant, then the constant A is 1, because we already know that the next value is equal to the previous value. We are lucky that we have a constant value in this example, but even if it has any other linear properties, we can assume it to be 1.

•The value H = 1, because we know that the measured value is composed of the state value and some noise. You rarely encounter situations where H is different from 1 in real life.

Finally, we assume we get the following measurements.

![alt text](19.png)

We should start somewhere, say $k = 0$. We should find or assume an initial state. Here we give some initial values. If we choose this way, it means that there is no noise in the environment, and this assumption will cause all the results of $X$ in the state of $k$ to be $0$ (retaining the initial state), so we generally do not take us to write time updates and measurements Update equation.  
$X_0 = 0 P_0 = 1 P_0 = 0$

![alt text](20.png)

Steps for each round of calculation:
![alt text](21.png)
![alt text](22.png)